## Selenium

* 安裝 [Selector Gadget](https://chrome.google.com/webstore/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb) 和 [Quick Javascript Switcher](https://chrome.google.com/webstore/detail/quick-javascript-switcher/geddoclleiomckbhadiaipdggiiccfje)
* 下載對應版本的 [chromedriver](https://chromedriver.chromium.org/downloads)
* 透過 Selector Gadget 觀察 target

In [ ]:
#coding=utf-8
import numpy as np
import pandas as pd
import random
from datetime import datetime 

from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib.request

import sqlalchemy as sa
import os, time
from IPython.display import clear_output

In [ ]:
min_time = 4
max_time = 8

In [ ]:
#不完整
input_city_nm = input('請輸入縣市名稱：')
if input_city_nm=='基隆市':
    city_num = 2
    section = 'all'
elif input_city_nm=='台北市':
    city_num = 1
elif input_city_nm=='新北市':
    city_num = 3
    section = [37,38]
elif input_city_nm=='桃園市':
    city_num = 6
elif input_city_nm=='新竹市':
    city_num = 4
elif input_city_nm=='台中市':
    city_num = 8
elif input_city_nm=='台南市':
    city_num = 15
    section = [206,207,208,209,210,211]
elif input_city_nm=='高雄市':
    city_num = 17
else:
    print('請確認輸入的縣市是否正確')

In [4]:
def get_href(url, first_row=0):
#     url = r'https://rent.591.com.tw/?region={}'.format(city_num)
    print(r'url：{}'.format(url))

    browser = webdriver.Chrome(executable_path=r'D:\GitHub_彙整\GitHub_2022_爬蟲_591租屋\chromedriver.exe')
    browser.get(url)
        
    time.sleep(random.randint(min_time, max_time) + abs(random.random()))

    nodes = []
    nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.switch-amount span')
    tot_num = int(nodes[0].text.replace(',',''))

    print(r'個數：{}'.format(int(nodes[0].text.replace(',', ''))))
        
#     first_row = 0
#     web_list = []

    url = r'{}&firstRow={}&totalRows={}'.format(url, city_num, first_row, tot_num)
    #browser = webdriver.Chrome(executable_path=r'D:\GitHub_彙整\GitHub_爬蟲\chromedriver.exe')
    browser.get(url)

    time.sleep(random.randint(min_time, max_time) + abs(random.random()))
    nodes = []
    nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.vue-list-rent-item a')
    
    hrefs = []
    for node in nodes:
        print(node.get_attribute('href'))
        if node.get_attribute('href') != None:
            hrefs = hrefs + [node.get_attribute('href')]
        hrefs = list(set(hrefs))
#       web_list = web_list + hrefs
    
    rec_num = len(hrefs)
    browser.close()
    return(hrefs, rec_num)

In [6]:
def web_crwal(hrefs, table = []):
    for i in range(0, len(hrefs)):
        try:
            browser = webdriver.Chrome(executable_path=r'D:\GitHub_彙整\GitHub_2022_爬蟲_591租屋\chromedriver.exe')
            browser.get(hrefs[i])

            node_data=[]
            #1
            print('第 {} 筆'.format(i+1))
            node_data.append(i+1)
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #2
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='#app li:nth-child(1) a')[0].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #3
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='#app li:nth-child(2) a')[0].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #4
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='.load-map')[0].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #5, 6
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='#rightConFixed .name')[0].text.replace(' ', '').split(':')[0])
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='#rightConFixed .name')[0].text.replace(' ', '').split(':')[1])
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #7
            try: 
    #         nodes = browser.find_elements(by=By.CSS_SELECTOR, value='#rightConFixed .tel-txt')
            #nodes = browser.find_elements(by=By.CSS_SELECTOR, value='#rightConFixed .white .reference')
    #         print('電話：{}'.format(nodes[0].text.replace(' ', '')))
                node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='#rightConFixed .tel-txt')[0].text.replace(' ', ''))
            except Exception as e:
                print('電話：')
                node_data.append('')
                pass
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #8
            try: 
                node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='#positionRound .ellipsis a')[1].text.replace(' ', ''))
            except IndexError: 
                node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='.address.ellipsis span')[0].text.replace(' ', ''))
            except Exception as e:
                node_data.append('')
                pass
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #9
    #     nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')
        #nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span:nth-child(1)')
    #     print('房型：{}'.format(nodes[0].text.replace(' ', '')))
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')[0].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #10
    #     nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')
        #nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span:nth-child(3)')
    #     print('坪數：{}'.format(nodes[2].text.replace(' ', '')))
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')[2].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #11
    #     nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')
        #nodes = browser.find_elements(by=By.CSS_SELECTOR, value='span:nth-child(5)')
    #     print('樓層：{}'.format(nodes[4].text.replace(' ', '')))
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')[4].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #12
    #     nodes = browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')
        #nodes = browser.find_elements(by=By.CSS_SELECTOR, value='span:nth-child(7)')
    #     print('樓型：{}'.format(nodes[6].text.replace(' ', '')))
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='.house-pattern span')[6].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #13
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='.house-price span')[0].text.replace(',', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #14
            node_data.append(browser.find_elements(by=By.CSS_SELECTOR, value='h1')[0].text.replace(' ', ''))
            time.sleep(random.randint(min_time, max_time) + abs(random.random()))

            #15
            node_data.append(hrefs[i])

            #16
            node_data.append(datetime.now().strftime('%y/%m/%d %H:%M:%S'))

            table.append(node_data)
    #         print('='*20)

            time.sleep(random.randint(min_time, max_time) + abs(random.random()))
            browser.close()
        except Exception as e:
            browser.close()
            pass

    return(table)

In [7]:
table = []
first_row = 0
if(section == 'all'):
    url = r'https://rent.591.com.tw/?region={}'.format(city_num)
    while True:
        a = get_href(url, first_row)
        b = web_crwal(a[0], table)

        rec_num = a[1]
        first_row = first_row + rec_num
        print(r'目前筆數：{}'.format(first_row))
        #if first_row >= tot_num:
        if first_row >= 30:
            break
else:
    for s in range(0, len(section)):
        url = r'https://rent.591.com.tw/?region={}&section={}&searchtype=1'.format(city_num, section[s])
        print(r'區域號碼：{}'.format(section[s]))
        while True:
            a = get_href(url, first_row)
            b = web_crwal(a[0], table)
     
            rec_num = a[1]
            first_row = first_row + rec_num
            print(r'目前筆數：{}'.format(first_row))
            #if first_row >= tot_num:
            if first_row >= 30:
                break            
table = b

區域號碼：37
url：https://rent.591.com.tw/?region=3&section=37&searchtype=1


<ipython-input-4-cd970b5cbe8b>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=r'D:\GitHub_彙整\GitHub_2022_爬蟲_591租屋\chromedriver.exe')


個數：638
https://rent.591.com.tw/rent-detail-13170204.html
https://rent.591.com.tw/rent-detail-13309755.html
https://rent.591.com.tw/rent-detail-13252257.html
https://rent.591.com.tw/rent-detail-13224844.html
https://rent.591.com.tw/rent-detail-13275392.html
https://rent.591.com.tw/rent-detail-13308344.html
https://rent.591.com.tw/rent-detail-13195122.html
None
https://rent.591.com.tw/rent-detail-13201115.html
https://rent.591.com.tw/rent-detail-13229140.html
https://rent.591.com.tw/rent-detail-13303079.html
https://rent.591.com.tw/rent-detail-13303823.html
None
https://rent.591.com.tw/rent-detail-13285236.html
https://rent.591.com.tw/rent-detail-13295094.html
https://rent.591.com.tw/rent-detail-13259293.html
None
https://rent.591.com.tw/rent-detail-13307256.html
https://rent.591.com.tw/rent-detail-13201674.html
https://rent.591.com.tw/rent-detail-13203939.html
https://rent.591.com.tw/rent-detail-13211636.html
https://rent.591.com.tw/rent-detail-13192305.html
https://rent.591.com.tw/rent

<ipython-input-6-d094738fe425>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=r'D:\GitHub_彙整\GitHub_2022_爬蟲_591租屋\chromedriver.exe')


第 1 筆
第 2 筆
第 3 筆
第 4 筆
第 5 筆
第 6 筆
第 7 筆
第 8 筆
第 9 筆
第 10 筆
第 11 筆
第 12 筆
第 13 筆
第 14 筆
第 15 筆
第 16 筆
第 17 筆
第 18 筆
第 19 筆
第 20 筆
第 21 筆
第 22 筆
第 23 筆
第 24 筆
第 25 筆
第 26 筆
第 27 筆
第 28 筆
第 29 筆
第 30 筆
目前筆數：30
區域號碼：38
url：https://rent.591.com.tw/?region=3&section=38&searchtype=1
個數：968
https://rent.591.com.tw/rent-detail-13278511.html
https://rent.591.com.tw/rent-detail-13228046.html
None
https://rent.591.com.tw/rent-detail-13167056.html
None
https://rent.591.com.tw/rent-detail-13256615.html
https://rent.591.com.tw/rent-detail-13285761.html
https://rent.591.com.tw/rent-detail-13275834.html
None
https://rent.591.com.tw/rent-detail-13244890.html
None
https://rent.591.com.tw/rent-detail-13274946.html
None
https://rent.591.com.tw/rent-detail-13313886.html
https://rent.591.com.tw/rent-detail-13274762.html
https://rent.591.com.tw/rent-detail-13246704.html
https://rent.591.com.tw/rent-detail-13234025.html
https://rent.591.com.tw/rent-detail-13274138.html
https://rent.591.com.tw/rent-detail-131

### 資料寫入 dataframe

In [8]:
node_df = pd.DataFrame(table)
# node_df['時間'] = datetime.now().strftime('%y/%m/%d %H:%M:%S')

In [9]:
node_df = node_df.rename(columns={0:'序號', 1:'縣市', 2:'行政區', 3:'地址', 4:'角色', 
                                  5:'姓名', 6:'電話', 7:'社區', 8:'房型', 9:'坪數', 
                                  10:'樓層', 11:'樓型', 12:'金額', 13:'主旨', 14:'網址',
                                  15:'時間'})
node_df.head()

,序號,縣市,行政區,地址,角色,姓名,電話,社區,房型,坪數,樓層,樓型,金額,主旨,網址,時間
0,1,新北市,永和區,永和區永和路一段,屋主,王阿姨(仲介勿擾),0919-522-855,,2房1廳1衛,20坪,2F/2F,透天厝,25000元/月,2房1廳1衛近頂溪樂華家具家齊全,https://rent.591.com.tw/rent-detail-13192305.html,22/09/30 10:58:35
1,2,新北市,永和區,永和區竹林路225巷,仲介,劉先生(收服務費),0976-105-031,,雅房,8坪,5F/5F,公寓,9800元/月,近永福橋/獨立陽台洗衣機/可貓大雅房,https://rent.591.com.tw/rent-detail-13306985.html,22/09/30 11:00:20
2,3,新北市,永和區,永和區民樂街69巷,屋主,顏小姐,0986-075-579,,3房2廳1衛,30坪,4F/5F,公寓,22900元/月,（已收訂金）謝謝,https://rent.591.com.tw/rent-detail-13196924.html,22/09/30 11:02:03
3,4,新北市,永和區,永和區保福路一段23巷,代理人,曾小姐/先生,0906-288-237,,獨立套房,8坪,5F/5F,公寓,11888元/月,頂溪站.樂華靜巷內.明亮舒適單人套房,https://rent.591.com.tw/rent-detail-13208409.html,22/09/30 11:03:50
4,5,新北市,永和區,永和區民有街,代理人,曾小姐/先生,0906-288-237,,2房1廳1衛,18坪,5F/5F,公寓,20000元/月,景平站.明亮舒適設計感兩房一廳,https://rent.591.com.tw/rent-detail-13178095.html,22/09/30 11:05:30


### 資料輸出

#### CSV檔

In [10]:
path = 'D:\GitHub_彙整\GitHub_2022_爬蟲_591租屋\Data'

In [12]:
node_df.to_csv(r'{}\591租屋_{}_完整.csv'.format(path, input_city_nm), index=False, sep='^')

#### Excel檔

In [11]:
Result_PATH = r'{}\591租屋_{}_完整_v2.xlsx'.format(path, input_city_nm)

# writing to Excel
student_result  = pd.ExcelWriter( Result_PATH, engine='xlsxwriter')

# write students data to excel
node_df.to_excel(student_result, sheet_name=input_city_nm, index=False, encoding='utf8')

# save the students result excel
student_result.save()
print('Students data is successfully written into Excel File')

Students data is successfully written into Excel File


#### 依條件篩選資料

In [14]:
df1 = pd.read_excel(r'{}\系統台分區.xlsx'.format(path))

In [24]:
df2 = node_df.merge(df1, how='inner', on=['縣市','行政區'])
df2.head()

,序號,縣市,行政區,地址,角色,姓名,電話,社區,房型,坪數,樓層,樓型,金額,主旨,網址,時間,系統,系統台區
0,6,基隆市,中正區,中正區新豐街162巷,屋主,張先生(仲介勿擾),0928-683-340,早安國揚,3房2廳2衛,44坪,3F/9F,電梯大樓,28000元/月,早安國揚正大三房+車位,https://rent.591.com.tw/rent-detail-13152790.html,22/09/26 14:50:56,A,AA
1,14,基隆市,中正區,中正區觀海街,仲介,陳小姐,0902-036-358,中正區觀海街,獨立套房,10坪,7F/16F,電梯大樓,6000元/月,📍兆基社宅.水美套房.低價搶租📍,https://rent.591.com.tw/rent-detail-13267199.html,22/09/26 15:03:11,A,AA
2,16,基隆市,中正區,中正區義一路,屋主,黃先生(仲介勿擾),0917-702-777,,分租套房,7坪,5F/6F,透天厝,8000元/月,基隆市中心/夜市旁/交通便利,https://rent.591.com.tw/rent-detail-13163396.html,22/09/26 15:06:18,A,AA
3,17,基隆市,中正區,中正區信六路43號,仲介,劉先生1筆,0905-256-020,,分租套房,5坪,4F/6F,公寓,6500元/月,市區大套房、獨立洗衣機、包水網路第四台,https://rent.591.com.tw/rent-detail-13214756.html,22/09/26 15:07:52,A,AA
4,23,基隆市,中正區,中正區新豐街,屋主,孔先生(仲介勿擾),0979-193-278,,獨立套房,7坪,3F/4F,公寓,5000元/月,新豐街優質套房歡迎看屋(C、D房),https://rent.591.com.tw/rent-detail-13176108.html,22/09/26 15:16:57,A,AA


In [25]:
df2.iloc[:, 1:15].to_excel(r'{}\591租屋_{}_需求.xlsx'.format(path, input_city_nm), index=False, encoding='utf8')

### 參考網址

* [Pandas 中 dataframe 常用操作](https://blog.csdn.net/Parzival_/article/details/114240650)
* [Pandas DataFrame 匯出到 Excel 檔案](https://www.delftstack.com/zh-tw/howto/python-pandas/export-pandas-dataframe-to-excel-file/)